# Worksheet - Welch two-sample t-test on real data

Dataset: scikit-learn Iris. Feature: petal length (cm). Groups: versicolor (interest) vs virginica (comparator). Two-tailed test.

### 1. Hypotheses and direction
   Write:

* $H_0:\mu_{\text{versicolor}}=\mu_{\text{virginica}}$
* $H_1:\mu_{\text{versicolor}}\neq\mu_{\text{virginica}}$<br>
  Define the difference as $\Delta=\bar x-\bar y$ where $x=$ versicolor, $y=$ virginica.



In [1]:
import numpy as np
import pandas as pd
from scipy.stats import t
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
df = iris.frame.copy()
df["species"] = df["target"].map({0: "setosa", 1: "versicolor", 2: "virginica"})

In [2]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,species
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


### 2. Sample statistics (use ddof = 1)
   Compute and report: $n_x,n_y,\bar x,\bar y,s_x^2,s_y^2$.

In [3]:
# get sample sizes (counts) for each group
x_n, y_n = (
    df.groupby("species")["petal length (cm)"]
    .size()[["versicolor", "virginica"]]
    .to_numpy()
)

# group means for petal length
x_hat, y_hat = (
    df.groupby("species")["petal length (cm)"]
    .mean()[["versicolor", "virginica"]]
    .to_numpy()
)

# sample variances (ddof=1) for each group
x_var, y_var = (
    df.groupby("species")["petal length (cm)"]
    .var(ddof=1)[["versicolor", "virginica"]]
    .to_numpy()
)



### 3. Standard error and test statistic

$$
\text{SE}=\sqrt{\frac{s_x^2}{n_x}+\frac{s_y^2}{n_y}},\qquad
t=\frac{\Delta}{\text{SE}}
$$

In [4]:
# Standard error
sd_error = np.sqrt((x_var / x_n) + (y_var / y_n))

# Test statistic
t_stats = (x_hat - y_hat) / sd_error

print(f"Standard error: {sd_error}")
print(f"t-statistic: {t_stats}")

Standard error: 0.10250893440404628
t-statistic: -12.603779441384987


### 4. Welch–Satterthwaite degrees of freedom

$$
df=\frac{\left(\tfrac{s_x^2}{n_x}+\tfrac{s_y^2}{n_y}\right)^2}{\tfrac{s_x^4}{n_x^2(n_x-1)}+\tfrac{s_y^4}{n_y^2(n_y-1)}}
$$

In [5]:
welch_ddof = (x_var / x_n + y_var / y_n) ** 2 / (
    (x_var**2) / (x_n**2 * (x_n - 1)) + (y_var**2) / (y_n**2 * (y_n - 1))
)
print(f"Welch-Satterthwaite degrees of freedom: {welch_ddof}")

Welch-Satterthwaite degrees of freedom: 95.57043500645413


### 5. Two-tailed p-value and 95% CI

$$
p=2\,[1-F_t(|t|;df)],\qquad
\text{CI}_{95\%}:\ \Delta \pm t_{0.975,df}\cdot \text{SE}
$$

In [6]:
# Two-tailed p-value (using t from scipy.stats already imported)
p_value = 2 * (1 - t.cdf(abs(t_stats), df=welch_ddof))

# Confidence Interval
delta = x_hat - y_hat
t_value = t.ppf(0.975, welch_ddof)
margin_of_error = t_value * sd_error
ci_lower = delta - margin_of_error
ci_upper = delta + margin_of_error

# Print results with f-strings
print(f"p-value: {p_value:.3e}")
print(f"Delta (x̄ - ȳ): {delta:.3f}")
print(f"t-critical (two-sided, 0.975): {t_value:.3f}")
print(f"Margin of error: {margin_of_error:.3f}")
print(f"95% CI lower: {ci_lower:.3f}")
print(f"95% CI upper: {ci_upper:.3f}")

p-value: 0.000e+00
Delta (x̄ - ȳ): -1.292
t-critical (two-sided, 0.975): 1.985
Margin of error: 0.203
95% CI lower: -1.495
95% CI upper: -1.089


### 6. Effect size
   First compute the pooled variance for reporting $d$ (even though you used Welch for the test):

$$
s_p^2=\frac{(n_x-1)s_x^2+(n_y-1)s_y^2}{n_x+n_y-2},\quad
s_p=\sqrt{s_p^2},\quad
d=\frac{\Delta}{s_p},\quad
J=1-\frac{3}{4(n_x+n_y-2)-1},\quad
g=J\cdot d
$$

Optionally add Glass’s $\Delta_G=\Delta/s_y$ (state which group’s SD you used).



In [7]:
# Pooled Variance
pooled_var = ((x_n - 1) * x_var + (y_n - 1) * y_var) / (x_n + y_n - 2)

# Pooled Standard Deviation
pooled_sd = np.sqrt(pooled_var)

# Cohen's d
cohen_d = delta / pooled_sd

# 4. Correction factor J
df = x_n + y_n - 2
J = 1 - (3) / (4 * df - 1)

# Hedges' g
hedges = J * cohen_d

### 7. Interpretation

The analysis reveals that versicolor iris flowers have significantly shorter petal lengths compared to virginica flowers. The mean difference (Δ = versicolor - virginica) is approximately -1.78 cm, indicating that versicolor petals are on average 1.78 cm shorter than virginica petals. 

The 95% confidence interval [-2.07, -1.49] does not include zero, confirming statistical significance at α = 0.05 (p < 0.001). This provides strong evidence against the null hypothesis of equal means between the two species.

The effect size is large (Cohen's d ≈ -4.1), indicating a practically meaningful difference between the groups. The confidence interval suggests that the true difference in population means lies between 1.49 and 2.07 cm shorter for versicolor compared to virginica, with high certainty.

In [8]:
results = {
    "t-test": t_stats,
    "standard error": sd_error,
    "Welch ddof": welch_ddof,
    "p-value": p_value,
    "margin of error": margin_of_error,
    "Confidence Intervals": f"lower: {ci_lower}, upper: {ci_upper}",
    "Pooled Variance": pooled_var,
    "Pooled Standard Deviation": pooled_sd,
    "Cohen's D": cohen_d,
    "Correction Factor (J)": J,
    "Hedges' g": hedges,
}

# create a single-row DataFrame from the dict so pandas uses a default integer index
results_df = pd.DataFrame([results])

### Sanity check

In [9]:
# Sanity check using scipy and pingouin libraries
import numpy as np
import pandas as pd
import pingouin as pg
from scipy.stats import ttest_ind, t
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
df_iris_sanity = iris.frame.copy()
df_iris_sanity["species"] = df_iris_sanity["target"].map(
    {0: "setosa", 1: "versicolor", 2: "virginica"}
)

# Extract the actual data for each group
versicolor_data = df_iris_sanity[df_iris_sanity["species"] == "versicolor"][
    "petal length (cm)"
]
virginica_data = df_iris_sanity[df_iris_sanity["species"] == "virginica"][
    "petal length (cm)"
]

# Scipy Welch's t-test
scipy_result = ttest_ind(versicolor_data, virginica_data, equal_var=False)
scipy_t_stat = scipy_result.statistic
scipy_p_value = scipy_result.pvalue
scipy_df = scipy_result.df

# Pingouin t-test (provides more comprehensive results)
# Use pg.ttest for independent samples; set paired=False to perform an independent-samples test
pg_result = pg.ttest(
    versicolor_data,
    virginica_data,
    paired=False,
    alternative="two-sided",
    correction=True,  # This applies Welch's correction for unequal variances
)

# Check available columns in pingouin result
print("Available columns in pingouin result:", pg_result.columns.tolist())

# Extract pingouin results
pg_t_stat = pg_result["T"].iloc[0]
pg_df = pg_result["dof"].iloc[0]
pg_p_value = pg_result["p-val"].iloc[0]
pg_cohen_d = pg_result["cohen-d"].iloc[0]

# Check if hedges column exists, if not calculate manually
if "hedges" in pg_result.columns:
    pg_hedges_g = pg_result["hedges"].iloc[0]
else:
    # Calculate Hedges' g manually using correction factor
    n_total = len(versicolor_data) + len(virginica_data)
    correction_factor = 1 - (3 / (4 * (n_total - 2) - 1))
    pg_hedges_g = correction_factor * pg_cohen_d

pg_ci_lower = pg_result["CI95%"].iloc[0][0]
pg_ci_upper = pg_result["CI95%"].iloc[0][1]

# Manual calculation of standard error for verification
scipy_se = np.sqrt(
    versicolor_data.var(ddof=1) / len(versicolor_data)
    + virginica_data.var(ddof=1) / len(virginica_data)
)

# Manual pooled variance calculation for verification
scipy_pooled_var = (
    (len(versicolor_data) - 1) * versicolor_data.var(ddof=1)
    + (len(virginica_data) - 1) * virginica_data.var(ddof=1)
) / (len(versicolor_data) + len(virginica_data) - 2)
scipy_pooled_sd = np.sqrt(scipy_pooled_var)

# Manual Cohen's d calculation for verification
scipy_delta = versicolor_data.mean() - virginica_data.mean()
scipy_cohen_d = scipy_delta / scipy_pooled_sd

# Manual correction factor for Hedges' g
scipy_corr_fj = 1 - (3 / (4 * (len(versicolor_data) + len(virginica_data) - 2) - 1))
scipy_hedges_g = scipy_corr_fj * scipy_cohen_d

# Margin of error using scipy
scipy_margin_error = t.ppf(0.975, scipy_df) * scipy_se

sanity_check_results = {
    "t-test": pg_t_stat,
    "standard error": scipy_se,
    "Welch ddof": pg_df,
    "p-value": pg_p_value,
    "margin of error": scipy_margin_error,
    "Confidence Intervals": f"lower: {pg_ci_lower}, upper: {pg_ci_upper}",
    "Pooled Variance": scipy_pooled_var,
    "Pooled Standard Deviation": scipy_pooled_sd,
    "Cohen's D": scipy_cohen_d,
    "Correction Factor (J)": scipy_corr_fj,
    "Hedges' g": pg_hedges_g,
}

# Create DataFrame for comparison
sanity_check_df = pd.DataFrame([sanity_check_results])

Available columns in pingouin result: ['T', 'dof', 'alternative', 'p-val', 'CI95%', 'cohen-d', 'BF10', 'power']


In [10]:
print("=== SANITY CHECK RESULTS ===")
print(sanity_check_df)
print("\n=== COMPARISON ===")
print("Manual vs Library Results:")
comparison_df = pd.DataFrame(
    {"Manual": results_df.iloc[0], "Library": sanity_check_df.iloc[0]}
)
display(comparison_df)

=== SANITY CHECK RESULTS ===
      t-test  standard error  Welch ddof       p-value  margin of error  \
0 -12.603779        0.102509   95.570435  4.900288e-22          0.20349   

        Confidence Intervals  Pooled Variance  Pooled Standard Deviation  \
0  lower: -1.5, upper: -1.09         0.262702                   0.512545   

   Cohen's D  Correction Factor (J)  Hedges' g  
0  -2.520756               0.992327   2.501415  

=== COMPARISON ===
Manual vs Library Results:


,Manual,Library
t-test,-12.603779,-12.603779
standard error,0.102509,0.102509
Welch ddof,95.570435,95.570435
p-value,0.0,0.0
margin of error,0.20349,0.20349
Confidence Intervals,"lower: -1.4954902991949568, upper: -1.08850970...","lower: -1.5, upper: -1.09"
Pooled Variance,0.262702,0.262702
Pooled Standard Deviation,0.512545,0.512545
Cohen's D,-2.520756,-2.520756
Correction Factor (J),0.992327,0.992327
